In [2]:
import numpy as np
from cpymad.madx import Madx
import json
import pickle
from datetime import datetime

In [3]:
## Listing elements for predicting their parameters [gradients,alignments]
def read_elements(elements_file):
    elements = open(elements_file,'r')
    element = elements.readlines()
    num = 0
    elements=[]
    for elem in element:
        num+=1
        elements.append(elem.strip())
    return elements, num

In [14]:
## Method to add errors to a MAD-X structure
def add_errors(madx_compiler,error,distribution):
    quads, number = read_elements('quads.txt')
    
    if distribution == 'Gauss':
        seed = np.random.randint(0,999999999)
        madx_compiler.input('eoption,seed='+str(seed)+',add=True;')
        
        for quad in quads:
            index = np.random.choice(52,2)
            madx_compiler.input('select,flag=error,class=quadrupole,pattern='+quad+';')
            madx_compiler.input('ealign,dx:='+str(error)+'*gauss(),dy:='+str(error)+'*gauss();')
    else:
        arr = np.concatenate((np.random.rand(26)**5*0.0001,-np.random.rand(26)**5*0.0001))
        np.random.shuffle(arr)

        for quad in quads:
            index = np.random.choice(52,2)
            madx_compiler.input('select,flag=error,class=quadrupole,pattern='+quad+';')
            madx_compiler.input('ealign,dx:='+str(arr[index[0]])+'*gauss(),dy:='+str(arr[index[1]])+'*gauss();')

        
    
    
                            
    madx_compiler.input('esave,file="C:\\Users\\r_mam\\DeepLearningProject\\machine_imperfections";')
    madx_compiler.input('select,flag=myerrortable, class=quadrupole;')
    madx_compiler.input('etable,table=errors_table;')
                            
    table = madx_compiler.table.errors_table
    
    return table
                            

In [17]:
## Using MAD-X to get beam orbits [x,y] with errors [gradients, alignments, rolls]
def structure_calculate(file="VEPP4M_full.txt",distribution=None):
    madx = Madx(stdout=False)
    madx.option(echo = False, warn = False, info = False, twiss_print = False)
    madx.call(file=file)
    madx.input('Beam, particle = electron, energy = 1.8;')
    madx.input('use,sequence=RING;')
    
    errors_table = add_errors(madx,error=0.000001,distribution=distribution)
    
    madx.input('select,flag=twiss,class=monitor;')
    twiss = madx.twiss(sequence='RING',centre = True,table='twiss',file="C:\\Users\\r_mam\\DeepLearningProject\\measure_123.txt")
    madx.input('readtable, file="C:\\Users\\r_mam\\DeepLearningProject\\measure_123.txt",table=twiss_short;')
    table = madx.table
        
    x = table.twiss_short.x
    y = table.twiss_short.y
    x_y = np.stack((x,y))
    
    dx_error = table.errors_table.dx
    dy_error = table.errors_table.dy
    dx_dy=np.stack((dx_error,dy_error))
    
    array_to_file = (x_y,dx_dy)
    
    return array_to_file

In [18]:
## Constructing a file with data [input_data, labels] in a matrix format
data = []
for i in range(50000):
    array = structure_calculate(distribution='Gauss')
    data.append(array)
    if np.mod(i,100)==0:
        print("Gauss " + str(i) +" time " + str(datetime.now().time()))
for i in range(50000):
    array = structure_calculate(distribution='Uniform')
    data.append(array)
    if np.mod(i,100)==0:
        print("Uniform " + str(i) +" time " + str(datetime.now().time()))
    

file = open('dataset.pkl','wb')
pickle.dump(data,file)
file.close()

Gauss 0 time 13:33:21.813806
Gauss 100 time 13:33:51.368963
Gauss 200 time 13:34:20.925870
Gauss 300 time 13:34:50.161793
Gauss 400 time 13:35:19.240232
Gauss 500 time 13:35:48.642263
Gauss 600 time 13:36:17.860897
Gauss 700 time 13:36:47.971536
Gauss 800 time 13:37:17.237212
Gauss 900 time 13:37:46.837033
Gauss 1000 time 13:38:16.513302
Gauss 1100 time 13:38:46.899283
Gauss 1200 time 13:39:18.446175
Gauss 1300 time 13:39:47.100259
Gauss 1400 time 13:40:15.811175
Gauss 1500 time 13:40:44.950109
Gauss 1600 time 13:41:14.062910
Gauss 1700 time 13:41:43.396004
Gauss 1800 time 13:42:12.877549
Gauss 1900 time 13:42:41.807900
Gauss 2000 time 13:43:11.081644
Gauss 2100 time 13:43:40.380311
Gauss 2200 time 13:44:10.254335
Gauss 2300 time 13:44:40.169114
Gauss 2400 time 13:45:11.765377
Gauss 2500 time 13:45:43.056037
Gauss 2600 time 13:46:13.242987
Gauss 2700 time 13:46:47.166636
Gauss 2800 time 13:47:18.520592
Gauss 2900 time 13:47:48.442003
Gauss 3000 time 13:48:18.844887
Gauss 3100 time 13:4

Gauss 25200 time 15:46:03.174726
Gauss 25300 time 15:46:32.671979
Gauss 25400 time 15:47:02.647065
Gauss 25500 time 15:47:32.033211
Gauss 25600 time 15:48:01.603783
Gauss 25700 time 15:48:31.172331
Gauss 25800 time 15:49:01.089538
Gauss 25900 time 15:49:31.136842
Gauss 26000 time 15:50:00.656070
Gauss 26100 time 15:50:30.457901
Gauss 26200 time 15:51:01.610307
Gauss 26300 time 15:51:31.037117
Gauss 26400 time 15:52:00.615036
Gauss 26500 time 15:52:30.222988
Gauss 26600 time 15:52:59.747068
Gauss 26700 time 15:53:29.342867
Gauss 26800 time 15:53:58.767536
Gauss 26900 time 15:54:32.839832
Gauss 27000 time 15:55:06.915470
Gauss 27100 time 15:55:41.353630
Gauss 27200 time 15:56:15.568026
Gauss 27300 time 15:56:48.041536
Gauss 27400 time 15:57:22.032069
Gauss 27500 time 15:57:53.395935
Gauss 27600 time 15:58:28.491138
Gauss 27700 time 15:58:59.997711
Gauss 27800 time 15:59:31.371507
Gauss 27900 time 16:00:02.700718
Gauss 28000 time 16:00:33.900059
Gauss 28100 time 16:01:06.080415
Gauss 2820

Uniform 100 time 17:57:44.318805
Uniform 200 time 17:58:15.387205
Uniform 300 time 17:58:46.457353
Uniform 400 time 17:59:17.041698
Uniform 500 time 17:59:48.967114
Uniform 600 time 18:00:19.310921
Uniform 700 time 18:00:50.833389
Uniform 800 time 18:01:23.230291
Uniform 900 time 18:01:55.693341
Uniform 1000 time 18:02:25.772893
Uniform 1100 time 18:02:58.652238
Uniform 1200 time 18:03:29.496159
Uniform 1300 time 18:04:00.795876
Uniform 1400 time 18:04:33.627318
Uniform 1500 time 18:05:06.305872
Uniform 1600 time 18:05:36.992099
Uniform 1700 time 18:06:08.637739
Uniform 1800 time 18:06:39.432591
Uniform 1900 time 18:07:10.491989
Uniform 2000 time 18:07:41.615694
Uniform 2100 time 18:08:13.772430
Uniform 2200 time 18:08:42.935854
Uniform 2300 time 18:09:13.041795
Uniform 2400 time 18:09:44.407241
Uniform 2500 time 18:10:15.729808
Uniform 2600 time 18:10:44.983389
Uniform 2700 time 18:11:16.984094
Uniform 2800 time 18:11:48.657331
Uniform 2900 time 18:12:18.544990
Uniform 3000 time 18:12

Uniform 23900 time 19:56:09.202359
Uniform 24000 time 19:56:38.606672
Uniform 24100 time 19:57:08.197959
Uniform 24200 time 19:57:37.558369
Uniform 24300 time 19:58:06.957521
Uniform 24400 time 19:58:36.304041
Uniform 24500 time 19:59:05.697584
Uniform 24600 time 19:59:35.137722
Uniform 24700 time 20:00:04.772085
Uniform 24800 time 20:00:34.316543
Uniform 24900 time 20:01:03.779423
Uniform 25000 time 20:01:33.116378
Uniform 25100 time 20:02:03.916816
Uniform 25200 time 20:02:33.550558
Uniform 25300 time 20:03:03.000675
Uniform 25400 time 20:03:32.475033
Uniform 25500 time 20:04:02.456638
Uniform 25600 time 20:04:31.748777
Uniform 25700 time 20:05:01.232316
Uniform 25800 time 20:05:30.618100
Uniform 25900 time 20:06:00.178741
Uniform 26000 time 20:06:29.574138
Uniform 26100 time 20:06:59.128085
Uniform 26200 time 20:07:28.496468
Uniform 26300 time 20:07:57.874574
Uniform 26400 time 20:08:27.309341
Uniform 26500 time 20:08:56.641372
Uniform 26600 time 20:09:25.912104
Uniform 26700 time 2

Uniform 47400 time 21:51:16.938577
Uniform 47500 time 21:51:47.301226
Uniform 47600 time 21:52:17.617951
Uniform 47700 time 21:52:48.629248
Uniform 47800 time 21:53:19.042445
Uniform 47900 time 21:53:48.798548
Uniform 48000 time 21:54:19.924692
Uniform 48100 time 21:54:50.684755
Uniform 48200 time 21:55:22.279567
Uniform 48300 time 21:55:54.067129
Uniform 48400 time 21:56:27.045432
Uniform 48500 time 21:56:59.514863
Uniform 48600 time 21:57:32.380041
Uniform 48700 time 21:58:05.117415
Uniform 48800 time 21:58:38.124560
Uniform 48900 time 21:59:10.512655
Uniform 49000 time 21:59:42.161866
Uniform 49100 time 22:00:13.987263
Uniform 49200 time 22:00:46.122159
Uniform 49300 time 22:01:18.062543
Uniform 49400 time 22:01:48.195951
Uniform 49500 time 22:02:18.803020
Uniform 49600 time 22:02:48.318132
Uniform 49700 time 22:03:18.108875
Uniform 49800 time 22:03:48.720926
Uniform 49900 time 22:04:19.399943


In [4]:
## Method to add errors to a MAD-X structure
quads, number = read_elements('quads.txt')
def add_error_to_1element(madx_compiler,error,distribution):
    
    if distribution == 'Gauss':
        seed = np.random.randint(0,999999999)
        madx_compiler.input('eoption,seed='+str(seed)+',add=False;')
        
#         quad = np.random.choice(quads)
        
        for quad in quads:
            madx_compiler.input('select,flag=error,class=quadrupole,pattern='+quad+';')
            # madx_compiler.input('ealign,dx:='+str(error)+'*gauss(),dy:='+str(error)+'*gauss();')
            madx_compiler.input('efcomp,dkn:={0,'+str(error)+'*gauss()};')
            # madx_compiler.input('efcomp,dkn:={0,'+str(error)+'};')
        
    else:
        seed = np.random.randint(0,999999999)
        madx_compiler.input('eoption,seed='+str(seed)+',add=False;')
        
#         quad = np.random.choice(quads)
        
        for quad in quads:
            madx_compiler.input('select,flag=error,class=quadrupole,pattern='+quad+';')
            # madx_compiler.input('ealign,dx:='+str(error)+'*gauss(),dy:='+str(error)+'*gauss();')
            madx_compiler.input('efcomp,dkn:={0,'+str(error)+'*ranf()};')
            # madx_compiler.input('efcomp,dkn:={0,'+str(error)+'};')

        
    
    
                            
#     madx_compiler.input('esave,file="C:\\Users\\r_mam\\DeepLearningProject\\machine_imperfections";')
    madx_compiler.input('select,flag=myerrortable, class=quadrupole;')
    madx_compiler.input('etable,table=errors_table;')
                            
    
    table = madx_compiler.table.errors_table.k1l
    quad_index = quads.index(quad)
    
#     quads_errors = np.zeros(len(quads))
#     quads_errors[quad_index] = table.k1l
#     quads_errors[quad_index] = 1
    
    return table
                            

In [5]:
madx = Madx(stdout=True)
madx.option(echo = True, warn = True, info = True, twiss_print = True)
madx.call(file="VEPP4M_full.txt")
madx.input('Beam, particle = electron, energy = 1.8;')
# madx.command.hkicker.clone('CTXn', L=0.07, KICK=0.00001)
madx.input('use,sequence=RING;')
madx.input('select,flag=twiss,class=monitor;')
twiss = madx.twiss(sequence='RING',centre = True,table='twiss',file="measure_1234.txt")
madx.input('readtable, file="measure_1234.txt",table=twiss_short;')
table = madx.table.twiss_short
ideal_betx = table.betx
ideal_bety = table.bety

In [13]:
## Using MAD-X to get beam orbits [x,y] with errors [gradients, alignments, rolls]
def structure_calculate1(file="VEPP4M_full.txt",distribution=None):
    madx = Madx(stdout=False)
    madx.option(echo = False, warn = False, info = False, twiss_print = False)
    madx.call(file=file)
    madx.input('Beam, particle = electron, energy = 1.8;')
    madx.input('use,sequence=RING;')
    
    quads_errors = add_error_to_1element(madx,error=0.0001,distribution=distribution)
    
    madx.input('select,flag=twiss,class=monitor;')
    twiss = madx.twiss(sequence='RING',centre = True,table='twiss',file="C:\\Users\\r_mam\\DeepLearningProject\\measure_123.txt")
    madx.input('readtable, file="C:\\Users\\r_mam\\DeepLearningProject\\measure_123.txt",table=twiss_short;')
    table = madx.table
        
    x = table.twiss_short.betx - ideal_betx
    y = table.twiss_short.bety - ideal_bety
    x_y = np.stack((x,y))
#     print(x)
#     print(quads_errors)
    
    
    array_to_file = (x_y,quads_errors)
    
    return array_to_file

In [14]:
## Constructing a file with data [input_data, labels] in a matrix format
data = []
for i in range(50000):
    array = structure_calculate1(distribution='Gauss')
    data.append(array)
    if np.mod(i,100)==0:
        print("Gauss " + str(i) +" time " + str(datetime.now().time()))
for i in range(50000):
    array = structure_calculate1(distribution='Uniform')
    data.append(array)
    if np.mod(i,100)==0:
        print("Uniform " + str(i) +" time " + str(datetime.now().time()))
    

file = open('dataset.pkl','wb')
pickle.dump(data,file)
file.close()

Gauss 0 time 21:31:07.532068
Gauss 100 time 21:31:39.180117
Gauss 200 time 21:32:10.560003
Gauss 300 time 21:32:42.793388
Gauss 400 time 21:33:13.404367
Gauss 500 time 21:33:43.274764
Gauss 600 time 21:34:11.954010
Gauss 700 time 21:34:40.533904
Gauss 800 time 21:35:09.034854
Gauss 900 time 21:35:37.704139
Gauss 1000 time 21:36:07.101555
Gauss 1100 time 21:36:36.462214
Gauss 1200 time 21:37:05.520189
Gauss 1300 time 21:37:34.648884
Gauss 1400 time 21:38:03.796779
Gauss 1500 time 21:38:33.721016
Gauss 1600 time 21:39:02.595608
Gauss 1700 time 21:39:32.522461
Gauss 1800 time 21:40:01.442243
Gauss 1900 time 21:40:31.025490
Gauss 2000 time 21:40:59.565980
Gauss 2100 time 21:41:28.874972
Gauss 2200 time 21:41:58.773912
Gauss 2300 time 21:42:29.327767
Gauss 2400 time 21:43:00.815750
Gauss 2500 time 21:43:31.329488
Gauss 2600 time 21:44:01.610687
Gauss 2700 time 21:44:35.609397
Gauss 2800 time 21:45:12.580706
Gauss 2900 time 21:45:48.788174
Gauss 3000 time 21:46:26.023927
Gauss 3100 time 21:4

Gauss 25200 time 23:41:40.538797
Gauss 25300 time 23:42:10.681124
Gauss 25400 time 23:42:40.569458
Gauss 25500 time 23:43:10.437719
Gauss 25600 time 23:43:40.388960
Gauss 25700 time 23:44:10.261343
Gauss 25800 time 23:44:40.313063
Gauss 25900 time 23:45:10.498558
Gauss 26000 time 23:45:40.470696
Gauss 26100 time 23:46:10.413595
Gauss 26200 time 23:46:42.746817
Gauss 26300 time 23:47:15.052430
Gauss 26400 time 23:47:46.819688
Gauss 26500 time 23:48:18.347327
Gauss 26600 time 23:48:49.749482
Gauss 26700 time 23:49:22.332117
Gauss 26800 time 23:49:55.050751
Gauss 26900 time 23:50:27.015282
Gauss 27000 time 23:50:56.775451
Gauss 27100 time 23:51:27.596209
Gauss 27200 time 23:51:57.628539
Gauss 27300 time 23:52:28.176135
Gauss 27400 time 23:52:58.796008
Gauss 27500 time 23:53:29.757586
Gauss 27600 time 23:53:59.527630
Gauss 27700 time 23:54:32.751083
Gauss 27800 time 23:55:06.326690
Gauss 27900 time 23:55:42.041909
Gauss 28000 time 23:56:15.840414
Gauss 28100 time 23:56:48.781762
Gauss 2820

Uniform 100 time 01:51:33.871411
Uniform 200 time 01:52:04.102078
Uniform 300 time 01:52:34.256675
Uniform 400 time 01:53:04.435110
Uniform 500 time 01:53:34.739072
Uniform 600 time 01:54:04.948061
Uniform 700 time 01:54:35.119643
Uniform 800 time 01:55:05.164306
Uniform 900 time 01:55:35.317665
Uniform 1000 time 01:56:05.323562
Uniform 1100 time 01:56:35.496170
Uniform 1200 time 01:57:05.634815
Uniform 1300 time 01:57:35.822994
Uniform 1400 time 01:58:06.087741
Uniform 1500 time 01:58:36.189556
Uniform 1600 time 01:59:06.314652
Uniform 1700 time 01:59:37.276487
Uniform 1800 time 02:00:07.783945
Uniform 1900 time 02:00:38.038818
Uniform 2000 time 02:01:08.365792
Uniform 2100 time 02:01:40.027511
Uniform 2200 time 02:02:10.525028
Uniform 2300 time 02:02:40.989446
Uniform 2400 time 02:03:11.476263
Uniform 2500 time 02:03:42.372257
Uniform 2600 time 02:04:12.486174
Uniform 2700 time 02:04:42.790303
Uniform 2800 time 02:05:12.968537
Uniform 2900 time 02:05:43.149845
Uniform 3000 time 02:06

Uniform 23900 time 03:52:00.591406
Uniform 24000 time 03:52:30.994548
Uniform 24100 time 03:53:01.066572
Uniform 24200 time 03:53:31.172067
Uniform 24300 time 03:54:01.249452
Uniform 24400 time 03:54:31.387209
Uniform 24500 time 03:55:01.578836
Uniform 24600 time 03:55:31.700193
Uniform 24700 time 03:56:01.714166
Uniform 24800 time 03:56:31.941317
Uniform 24900 time 03:57:02.102458
Uniform 25000 time 03:57:32.260169
Uniform 25100 time 03:58:02.349063
Uniform 25200 time 03:58:32.568166
Uniform 25300 time 03:59:02.889758
Uniform 25400 time 03:59:33.732761
Uniform 25500 time 04:00:03.907655
Uniform 25600 time 04:00:34.031980
Uniform 25700 time 04:01:04.123242
Uniform 25800 time 04:01:34.263444
Uniform 25900 time 04:02:04.363534
Uniform 26000 time 04:02:35.966671
Uniform 26100 time 04:03:06.865477
Uniform 26200 time 04:03:37.072666
Uniform 26300 time 04:04:07.267928
Uniform 26400 time 04:04:37.368089
Uniform 26500 time 04:05:07.648643
Uniform 26600 time 04:05:37.817109
Uniform 26700 time 0

Uniform 47400 time 05:51:15.539371
Uniform 47500 time 05:51:45.908545
Uniform 47600 time 05:52:16.225167
Uniform 47700 time 05:52:46.422236
Uniform 47800 time 05:53:16.533218
Uniform 47900 time 05:53:46.742635
Uniform 48000 time 05:54:16.858102
Uniform 48100 time 05:54:47.079048
Uniform 48200 time 05:55:17.255473
Uniform 48300 time 05:55:47.516427
Uniform 48400 time 05:56:17.635810
Uniform 48500 time 05:56:47.872292
Uniform 48600 time 05:57:18.125432
Uniform 48700 time 05:57:48.453171
Uniform 48800 time 05:58:18.944815
Uniform 48900 time 05:58:49.119572
Uniform 49000 time 05:59:19.205521
Uniform 49100 time 05:59:50.256760
Uniform 49200 time 06:00:20.343641
Uniform 49300 time 06:00:50.609711
Uniform 49400 time 06:01:20.856923
Uniform 49500 time 06:01:51.167945
Uniform 49600 time 06:02:22.489438
Uniform 49700 time 06:02:52.623759
Uniform 49800 time 06:03:23.069707
Uniform 49900 time 06:03:53.353768


In [44]:
# file1 = open('dataset.pkl','rb')
# file2 = pickle.load(file1)
# print(len(file2))

100


In [7]:
## Method to add errors to a MAD-X structure
def add_errors_to_element(madx_compiler,error,distribution):
    quads, number = read_elements('quads.txt')
    
    if distribution == 'Gauss':
        seed = np.random.randint(0,999999999)
        madx_compiler.input('eoption,seed='+str(seed)+',add=True;')
        
        
        index = np.random.choice(52,2)
        quad = np.random.choice(quads)
        
        madx_compiler.input('select,flag=error,class=quadrupole,pattern='+quad+';')
        madx_compiler.input('ealign,dx:='+str(error)+'*gauss(),dy:='+str(error)+'*gauss();')
    else:
        arr = np.concatenate((np.random.rand(26)**5*0.0001,-np.random.rand(26)**5*0.0001))
        np.random.shuffle(arr)


        index = np.random.choice(52,2)
        quad = np.random.choice(quads)
        
        madx_compiler.input('select,flag=error,class=quadrupole,pattern='+quad+';')
        madx_compiler.input('ealign,dx:='+str(arr[index[0]])+'*gauss(),dy:='+str(arr[index[1]])+'*gauss();')

        
    
    
                            
    madx_compiler.input('esave,file="C:\\Users\\r_mam\\DeepLearningProject\\machine_imperfections";')
    madx_compiler.input('select,flag=myerrortable, class=quadrupole;')
    madx_compiler.input('etable,table=errors_table;')
                            
    table = madx_compiler.table.errors_table
    quad_index = quads.index(quad)
    
    return quad,quad_index
                            

In [8]:
## Using MAD-X to get beam orbits [x,y] with errors [gradients, alignments, rolls]
def structure_calculate_for_one_element(file="VEPP4M_full.txt",distribution=None):
    madx = Madx(stdout=False)
    madx.option(echo = False, warn = False, info = False, twiss_print = False)
    madx.call(file=file)
    madx.input('Beam, particle = electron, energy = 1.8;')
    madx.input('use,sequence=RING;')
    
    quad,quad_index = add_errors_to_element(madx,error=0.000001,distribution=None)
    
    madx.input('select,flag=twiss,class=monitor;')
    twiss = madx.twiss(sequence='RING',centre = True,table='twiss',file="C:\\Users\\r_mam\\DeepLearningProject\\measure_123.txt")
    madx.input('readtable, file="C:\\Users\\r_mam\\DeepLearningProject\\measure_123.txt",table=twiss_short;')
    table = madx.table
        
    x = table.twiss_short.x
    y = table.twiss_short.y
    x_y = np.stack((x,y))
    
    
    array_to_file = (x_y,quad,quad_index)
    
    return array_to_file

In [10]:
## Constructing a file with data [input_data, labels] in a matrix format
data = []
for i in range(100000):
    array = structure_calculate_for_one_element(distribution='Gauss')
    data.append(array)
for i in range(100000):
    array = structure_calculate_for_one_element(distribution='Uniform')
    data.append(array)
    

file = open('dataset_one_element.pkl','wb')
pickle.dump(data,file)
file.close()

In [94]:
# a = ['a','b']
# ch = 'b'
# c = a.index(ch)
# print(c)

1
